In [15]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import soundfile as sf

In [2]:
DATASET_PATH = "../data/Zoom/Raw Data"

In [3]:
def extract_label(filepath):
    label = os.path.basename(filepath)
    return label

In [18]:
def load_wav_files(dataset_path):
    audio_data = []
    labels = [] 

    for root, _, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(".wav"):
                file_path = os.path.join(root, file)
                try:
                    signal, sr = librosa.load(file_path, sr=None)
                    audio_data.append(signal)
                    label = extract_label(file_path)
                    labels.append(label)
                except Exception as e:
                    print(f"Error loading {file_path}: {e}")
    
    return audio_data, labels

audio_data, labels = load_wav_files(DATASET_PATH)
print(f"Loaded {len(audio_data)} audio files.")

Loaded 70 audio files.


In [21]:
def isolate_keystrokes(audio, sr, threshold, segment_length=14400, n_fft=1024, hop_length=256):
    """
    Isolate keystrokes from audio using Short-Time Fourier Transform (STFT).
    Parameters:
    - audio: The audio signal to process.
    - sr: Sample rate of the audio.
    - threshold: Energy threshold for detecting keystrokes.
    - segment_length: Length of the segment to extract around each keystroke.
    - n_fft: Number of FFT components.
    - hop_length: Number of samples between frames.
    Returns:
    - segments: List of isolated keystroke segments.
    """
    # Compute the Fast Fourier Transform
    stft = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)
    # Sum the magnitude of the STFT across frequency bins to get energy
    energy = np.sum(np.abs(stft), axis=0)
    # Choose frames based on threashold
    keystroke_frames = np.where(energy > threshold)[0]

    # Group contiguous frames
    groups = np.split(keystroke_frames, np.where(np.diff(keystroke_frames) > 1)[0] + 1)
    
    # Take keystroke as center. Extract segment centered on the keystroke
    segments = []
    for group in groups:
        center_frame = int(np.mean(group))
        start = max((center_frame * hop_length) - segment_length // 2, 0)
        end = start + segment_length
        if end > len(audio):
            start = len(audio) - segment_length
            end = len(audio)
        segments.append(audio[start:end])
    return segments

In [ ]:
def zoom_keystroke_threshold(audio, sr, initial_threshold=100, step=5, target_keystrokes=25):
    """
    Implements Algorithm 1: Zoom Keystroke Threshold Setting.
    
    Parameters:
    - audio: 1D numpy array of the audio signal.
    - sr: Sample rate.
    - initial_threshold: Initial energy threshold.
    - step: Step size for adjusting threshold.
    - target_keystrokes: Desired number of keystrokes.
    
    Returns:
    - Optimal threshold value.
    """
    P = initial_threshold
    while True:
        S = isolate_keystrokes(audio, sr, threshold=P)
        num_keystrokes = len(S)
        if num_keystrokes == target_keystrokes:
            break
        elif num_keystrokes < target_keystrokes:
            P -= step
        elif num_keystrokes > target_keystrokes:
            P += step
        step *= 0.99 
    return P, S

In [20]:
def time_shift(segment, max_shift_percentage=0.4):
    """
    Shift the audio segment in time by a random amount.
    Parameters:
    - segment: The audio segment to shift.
    - max_shift_percentage: Maximum percentage of the segment length to shift.
    Returns:
    - shifted_segment: The time-shifted audio segment.
    """
    # Calculate the maximum shift in samples
    max_shift = int(len(segment) * max_shift_percentage)
    # Take a random integer shift within the range [-max_shift, max_shift]
    shift = np.random.randint(-max_shift, max_shift)
    # Shift the segment
    shifted_segment = np.roll(segment, shift)
    # Zero out the wrapped portion to avoid artifacts
    if shift > 0:
        shifted_segment[:shift] = 0
    elif shift < 0:
        shifted_segment[shift:] = 0
    return shifted_segment

In [ ]:
def generate_mel_spectrogram(segment, sr, n_mels=64, n_fft=1024, hop_length=225):
    """
    Generate a Mel spectrogram from the audio segment.
    Parameters:
    - segment: The audio segment to process.
    - sr: Sample rate of the audio.
    - n_mels: Number of Mel bands to generate.
    - n_fft: Number of FFT components.
    - hop_length: Number of samples between frames.
    Returns:
    - S_dB: The Mel spectrogram in decibels.
    """
    S = librosa.feature.melspectrogram(y=segment, sr=sr, n_fft=n_fft,  hop_length=hop_length, n_mels=n_mels)
    S_dB = librosa.power_to_db(S, ref=np.max)
    return S_dB


In [ ]:
def spec_augment(spectrogram, time_mask_percentage=0.1, freq_mask_percentage=0.1):
    """
    Augment the Mel spectrogram.
    Parameters:
    - spectrogram: The Mel spectrogram to augment.
    - time_mask_percentage: Percentage of the time dimension to mask.
    - freq_mask_percentage: Percentage of the frequency dimension to mask.
    Returns:
    - augmented_spec: The augmented Mel spectrogram.
    """
    augmented_spec = spectrogram.copy()
    n_mels, n_time = augmented_spec.shape
    # Calculate the lengths of the masks
    time_mask_length = max(1, int(n_time * time_mask_percentage))
    freq_mask_length = max(1, int(n_mels * freq_mask_percentage))
    # Randomly select the starting points for the masks
    time_start = np.random.randint(0, n_time - time_mask_length + 1)
    freq_start = np.random.randint(0, n_mels - freq_mask_length + 1)
    # Calculate the mean value of the spectrogram to use as the mask value
    mask_value = np.mean(augmented_spec)
    # Apply the time mask and frequency mask
    augmented_spec[freq_start:freq_start + freq_mask_length, time_start:time_start + time_mask_length] = mask_value
    return augmented_spec

In [ ]:
def process_audio_file(file_path, energy_threshold, output_dir, segment_length=14400, sr=44100):
    """
    Process a single audio file to isolate keystrokes and save the segments.
    Parameters:
    - file_path: Path to the audio file.
    - energy_threshold: Energy threshold for isolating keystrokes.
    - output_dir: Directory to save the processed segments.
    - segment_length: Length of the segment to extract around each keystroke.
    - sr: Sample rate for loading the audio.
    Returns:
    - sample_paths: List of paths to the saved audio segments.
    """
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    # Load the audio file
    audio, sr = librosa.load(file_path, sr=sr)
    segments = isolate_keystrokes(audio, sr, threshold=energy_threshold, segment_length=segment_length)
    
    sample_paths = []
    for idx, segment in enumerate(segments):
        # Apply time shift for data augmentation
        augmented_segment = time_shift(segment)
        # Save the raw audio segment (can be used for further processing or as input to a model)
        segment_path = os.path.join(output_dir, f"keystroke_{idx}.wav")
        sf.write(segment_path, augmented_segment, sr)
        sample_paths.append(segment_path)
        
        # Optionally, generate and save the mel-spectrogram image
        mel_spec = generate_mel_spectrogram(augmented_segment, sr)
        mel_spec_aug = spec_augment(mel_spec)
        plt.figure(figsize=(4, 4))
        librosa.display.specshow(mel_spec_aug, sr=sr, x_axis='time', y_axis='mel', cmap='viridis')
        plt.axis('off')
        image_path = os.path.join(output_dir, f"keystroke_{idx}.png")
        plt.savefig(image_path, bbox_inches='tight', pad_inches=0)
        plt.close()
    return sample_paths

In [ ]:
file_path = DATASET_PATH + '/0.wav'
energy_threshold = 100
output_directory = 'keystroke_samples'
    
sample_files = process_audio_file(file_path, energy_threshold, output_directory)
print(f"Extracted and saved {len(sample_files)} keystroke samples in '{output_directory}'")

Extracted and saved 30 keystroke samples in 'keystroke_samples'
